In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
from MulticoreTSNE import MulticoreTSNE as TSNE
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import csv
import gzip
import anndata as ad
from pathlib import Path
import glob

sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.set_figure_params(dpi=150, fontsize=10, dpi_save=600)

-----
anndata     0.8.0
scanpy      1.9.1
-----
MulticoreTSNE               NA
PIL                         9.1.0
appnope                     0.1.2
asttokens                   NA
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
cffi                        1.15.0
cloudpickle                 2.1.0
cycler                      0.10.0
cython_runtime              NA
cytoolz                     0.11.0
dask                        2022.9.0
dateutil                    2.8.2
debugpy                     1.5.1
decorator                   5.1.1
defusedxml                  0.7.1
entrypoints                 0.4
executing                   0.8.3
fsspec                      2022.8.2
h5py                        3.7.0
hypergeom_ufunc             NA
igraph                      0.10.1
ipykernel                   6.9.1
ipython_genutils            0.2.0
ipywidgets                  7.6.5
jedi                        0.18.1
jinja2                      3.0.3
joblib   

In [2]:
def calculator(sample):

    # STEP 1 -> set corresponding folders which the results of each specimen are written to
    output_folder = "/Users/lidiayung/project/resource/specimens"
    resource_folder = "/Users/lidiayung/project/resource/GSE174554_RAW/"
 
    matrix_path = glob.glob(f"{resource_folder}/GSM*_{sample}_matrix.mtx.gz")[0]
    features_path = glob.glob(f"{resource_folder}/GSM*_{sample}_features.tsv.gz")[0]
    barcodes_path = glob.glob(f"{resource_folder}/GSM*_{sample}_barcodes.tsv.gz")[0]


    output_path = os.path.join(output_folder, sample)
    # End of Step 1

    # STEP 2 -> creat output documents
    # path to unfiltered loom file (this will be created in the optional steps below)
    f_loom_path_unfilt = "unfiltered.loom" # test dataset, n=500 cells
    f_loom_path_scenic = "filtered_scenic.loom"
    f_anndata_path = "anndata.h5ad"
    f_pyscenic_output = "output.loom"
    f_final_loom = 'scenic_integrated-output.loom'

    # End of Step 2

    # Step 3-> read data into anndata
    mat = scipy.io.mmread(matrix_path)
    feature_ids = [row[0] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]
    gene_names = [row[0] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]
    feature_types = [row[0] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]
    barcodes = [row[0] for row in csv.reader(gzip.open(barcodes_path, mode="rt"), delimiter="\t")]
    matrix = pd.DataFrame.sparse.from_spmatrix(mat)
    matrix.columns = barcodes
    matrix=matrix.transpose() 
    matrix.columns = gene_names

    # convert the index and columns to DataFrame objects
    obs_df = matrix.index.to_frame(index=False)
    var_df = matrix.columns.to_frame(index=False)

    adata = ad.AnnData(X=matrix.values, obs=obs_df, var=var_df)


    row_attrs = {  "Gene": np.array(var_df[0]) ,}
    col_attrs = { "CellID":  np.array(matrix.index) , 
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
    }
    lp.create( f_loom_path_unfilt, adata.X.transpose(), row_attrs, col_attrs )   

    # End of Step 3

    #Step 4-> Filtering cells that have >2.5% mitochondrial read counts and <200 expressed genes

    adata = sc.read_loom( f_loom_path_unfilt )
    nCountsPerGene = np.sum(adata.X, axis=0)
    nCellsPerGene = np.sum(adata.X>0, axis=0)
        
    nCells=adata.X.shape[0]

    mito_genes = adata.var_names.str.startswith('MT-')
    # for each cell compute fraction of counts in mito genes vs. all genes
    adata.obs['percent_mito'] = np.sum(
    adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
    # add the total counts per cell as observations-annotation to adata
    adata.obs['n_counts'] = adata.X.sum(axis=1).A1
    
    sc.pp.filter_cells(adata, min_genes=200 )
    adata = adata[adata.obs['percent_mito'] <=0.025, :]
    #End of Step 4

    #Step 5 -> Read filtered data
    adata.write( f_anndata_path )
    # create basic row and column attributes for the loom file:
    row_attrs = {    "Gene": np.array(adata.var_names) ,}
    col_attrs = {
     "CellID": np.array(adata.obs_names) ,
     "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
     "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
     }
    lp.create( f_loom_path_scenic, adata.X.transpose(), row_attrs, col_attrs)
    
    #End of Step 5

    #Step 6 -> integrate metadata
    file = "/Users/lidiayung/project/resource/GSE174554_Tumor_normal_metadata.txt"
    metadata= pd.read_csv(file,sep=' ')
    metadata.head()
    #End of Step 6

    #Step 7 -> Calculate percentage and return the value from both pre and post data
    new_df = metadata[metadata["Sample#"] == sample].copy()
    new_df['Barcode'] = new_df['Barcode'].astype(str) + '-1'
    intersection_barcodes = set(new_df['Barcode']).intersection(adata.obs.index)
    tumor = new_df[new_df['Barcode'].isin(intersection_barcodes) & (new_df['Tumor_Normal_annotation'] == 'Tumor')]
    percentage = "{:.2%}".format(len(tumor)/len(adata.obs.nUMI))
    #print("Percentage: {:.2%}".format(len(tumor)/len(adata.obs.nUMI)))
    post_filtering = f"{len(tumor)}/{len(adata.obs.nUMI) - len(tumor)}"

    #End of Step 7


    sample_str = f"{post_filtering}"

    return (sample_str, percentage)

In [7]:
samples = ['SF9871', 'SF10432']
results = []
for sample in samples:
    obtainpercentage, post_filtering = calculator(sample)
    results.append((sample, obtainpercentage, post_filtering))

# Create a pandas DataFrame with the results
df = pd.DataFrame(results, columns=['Sample', 'Post-filtering','Percentage'])
print(df)

/var/folders/qq/n8l4bgxs53j_s3twq5jzc7g00000gn/T/ipykernel_87114/1974796303.py:40: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X=matrix.values, obs=obs_df, var=var_df)
/Users/lidiayung/miniconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/var/folders/qq/n8l4bgxs53j_s3twq5jzc7g00000gn/T/ipykernel_87114/1974796303.py:40: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X=matrix.values, obs=obs_df, var=var_df)

filtered out 541 cells that have less than 200 genes expressed
    Sample Post-filtering Percentage
0   SF9871         6/3592      0.17%
1  SF10432         434/19     95.81%


In [9]:
#not matching samples
samples = ['SF10099','SF11587','SF11720','SF11857','SF12704','SF4209','SF4810','SF6186','SF7307','SF9715','SF9962']
#'SF10022',
for sample in samples:
    obtainpercentage, post_filtering = calculator(sample)
    results.append((sample, obtainpercentage, post_filtering))

# Create a pandas DataFrame with the results
df_unmatching = pd.DataFrame(results, columns=['Sample', 'Post-filtering','Percentage'])
print(df_unmatching)

/var/folders/qq/n8l4bgxs53j_s3twq5jzc7g00000gn/T/ipykernel_87114/1974796303.py:40: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X=matrix.values, obs=obs_df, var=var_df)
/Users/lidiayung/miniconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/var/folders/qq/n8l4bgxs53j_s3twq5jzc7g00000gn/T/ipykernel_87114/1974796303.py:40: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X=matrix.values, obs=obs_df, var=var_df)

filtered out 1616 cells that have less than 200 genes expressed


/var/folders/qq/n8l4bgxs53j_s3twq5jzc7g00000gn/T/ipykernel_87114/1974796303.py:40: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X=matrix.values, obs=obs_df, var=var_df)
/Users/lidiayung/miniconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


     Sample Post-filtering Percentage
0    SF9871         6/3592      0.17%
1   SF10432         434/19     95.81%
2   SF10099        437/116     79.02%
3   SF11587        1252/47     96.38%
4   SF11720        630/906     41.02%
5   SF11857       157/3236      4.63%
6   SF12704       1189/639     65.04%
7    SF4209        633/102     86.12%
8    SF4810         499/27     94.87%
9    SF6186       2322/587     79.82%
10   SF7307         153/46     76.88%
11   SF9715          4/214      1.83%
12   SF9962       2660/808     76.70%


In [15]:
def prefilter(sample):
    file = "/Users/lidiayung/project/resource/GSE174554_Tumor_normal_metadata.txt"
    metadata= pd.read_csv(file,sep=' ')
    metadata.head()

    new_df = metadata[metadata["Sample#"] == sample].copy()
    tumor = new_df[(new_df['Tumor_Normal_annotation'] == 'Tumor')]
    percent = "{:.2%}".format(len(tumor)/len(new_df))
    
    return percent


In [25]:
samples = ['SF9871','SF10432','SF10099','SF11587','SF11720','SF11857','SF12704','SF4209','SF4810','SF6186','SF7307','SF9715','SF9962']
results = []
for sample in samples:
    calper = prefilter(sample)
    results.append((sample, calper))

# Create a pandas DataFrame with the results
df_prefilter = pd.DataFrame(results, columns=['Sample', 'percent'])
print(df_prefilter)


     Sample percent
0    SF9871   0.17%
1   SF10432  88.23%
2   SF10099  79.05%
3   SF11587  96.38%
4   SF11720  40.99%
5   SF11857   4.62%
6   SF12704  68.95%
7    SF4209  85.46%
8    SF4810  93.22%
9    SF6186  79.86%
10   SF7307  85.79%
11   SF9715  39.37%
12   SF9962  76.70%


In [10]:
#unmatching 2 batches 
samples = ['SF9715v2']
for sample in samples:
    obtainpercentage, post_filtering = calculator(sample)
    results.append((sample, obtainpercentage, post_filtering))

# Create a pandas DataFrame with the results
df_unmatching_v2 = pd.DataFrame(results, columns=['Sample', 'Post-filtering','Percentage'])
print(df_unmatching_v2)

/var/folders/qq/n8l4bgxs53j_s3twq5jzc7g00000gn/T/ipykernel_87114/1974796303.py:40: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X=matrix.values, obs=obs_df, var=var_df)
/Users/lidiayung/miniconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


      Sample Post-filtering Percentage
0     SF9871         6/3592      0.17%
1    SF10432         434/19     95.81%
2    SF10099        437/116     79.02%
3    SF11587        1252/47     96.38%
4    SF11720        630/906     41.02%
5    SF11857       157/3236      4.63%
6    SF12704       1189/639     65.04%
7     SF4209        633/102     86.12%
8     SF4810         499/27     94.87%
9     SF6186       2322/587     79.82%
10    SF7307         153/46     76.88%
11    SF9715          4/214      1.83%
12    SF9962       2660/808     76.70%
13  SF9715v2         0/3731      0.00%


In [11]:
#matchings
samples = ['SF10433', 'SF10592', 'SF6118v2', 'SF3996', 'SF12408', 'SF5581', 'SF3391', 'SF12774', 'SF9372', 'SF11248', 'SF2501', 'SF2628', 'SF12115', 'SF6621', 'SF11331', 'SF1199', 'SF9358', 'SF12165', 'SF4297', 'SF3073', 'SF9791', 'SF6118', 'SF7388', 'SF6996', 'SF6809', 'SF12460', 'SF11780', 'SF11082', 'SF9715v2', 'SF11916', 'SF12407', 'SF9510', 'SF4449', 'SF11488', 'SF12754', 'SF1343', 'SF11815', 'SF9494', 'SF7062', 'SF2979', 'SF10514', 'SF10857', 'SF11981', 'SF12333', 'SF12704v2', 'SF6098', 'SF12008', 'SF3448', 'SF10432', 'SF3243', 'SF7025', 'SF10565', 'SF10484', 'SF11977', 'SF3076', 'SF12382', 'SF4324', 'SF2990', 'SF4849', 'SF12427', 'SF12616', 'SF12751', 'SF9798', 'SF10441', 'SF10108', 'SF12243', 'SF11344', 'SF12707', 'SF8963', 'SF12594', 'SF9871', 'SF2777', 'SF11873']
for sample in samples:
    obtainpercentage, post_filtering = calculator(sample)
    results.append((sample, obtainpercentage, post_filtering))

# Create a pandas DataFrame with the results
df_matching = pd.DataFrame(results, columns=['Sample', 'Post-filtering','Percentage'])
print(df_matching)

/var/folders/qq/n8l4bgxs53j_s3twq5jzc7g00000gn/T/ipykernel_87114/1974796303.py:40: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X=matrix.values, obs=obs_df, var=var_df)
/Users/lidiayung/miniconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


filtered out 1 cells that have less than 200 genes expressed


/var/folders/qq/n8l4bgxs53j_s3twq5jzc7g00000gn/T/ipykernel_87114/1974796303.py:40: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X=matrix.values, obs=obs_df, var=var_df)
/Users/lidiayung/miniconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/var/folders/qq/n8l4bgxs53j_s3twq5jzc7g00000gn/T/ipykernel_87114/1974796303.py:40: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X=matrix.values, obs=obs_df, var=var_df)

IndexError: list index out of range